In [97]:
import os
import bs4
import json
import regex
import random
import urllib3
import certifi
from time import sleep
from bs4 import BeautifulSoup

In [27]:
url = "https://theatregratuit.com/"

In [5]:
http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())

In [105]:
def get_soup(url):
    r = http.request('GET', url)
    return BeautifulSoup(r.data, 'html.parser')

In [106]:
def get_page(url):
    r = http.request('GET', url)
    return r.data

In [47]:
def find_next_page_link(all_a_with_href, index):
    page_re = regex.compile(f"page.*{index}", regex.IGNORECASE)    
    for a in reversed(all_a_with_href):
        if regex.search(page_re, a.text):
            print("found link", a.text, a["href"])
            return a["href"]
    return False

In [50]:
def find_all_pages(url):
    index = 1
    pages = []
    pages.append(get_soup(url))
    all_a_with_href = pages[0].find_all("a", href=True)
    index += 1            
    next_link = find_next_page_link(all_a_with_href, index)
    while next_link:
        new_page = get_soup(f"{url}/{next_link}")
        pages.append(new_page)        
        all_a_with_href.extend(new_page.find_all("a", href=True))
        index += 1        
        next_link = find_next_page_link(all_a_with_href, index)
    return pages, all_a_with_href

In [51]:
pages, all_a_with_href = find_all_pages(url)

found link ALLER A LA PAGE 2 page2.html
found link ALLER A LA PAGE 3 page3.html


In [107]:
out_dir = 'theatregratuit-epubs'
if not os.path.isdir(out_dir):
    os.mkdir(out_dir)

In [115]:
for a in all_a_with_href:
    if a.text == 'epub':
        fname = a["href"].split('/')[-1]
        print("getting epub:", fname)
        epub = get_page(a['href'])
        with open(os.path.join(out_dir,fname), 'wb') as o:
            o.write(epub)
        sleep(0.5)

getting epub: count-959055-Un_chapeau_de_paille_dItalie_-_Eugene_Labiche.epub
getting epub: count-958959-La_Jarre_-_Luigi_Pirandello.epub
getting epub: count-958965-La_propriete_-_Charles_Cros.epub
getting epub: count-958992-Les_Boulingrin_-_Georges_Courteline.epub
getting epub: count-958771-LOURS_-_Anton_Tchekhov.epub
getting epub: count-959005-LOrphelin_de_la_Chine_-_Voltaire.epub
getting epub: count-959014-On_purge_bebe__-_Georges_Feydeau.epub
getting epub: count-958980-Le_gendre_de_Monsieur_Poirier_-_Emile_Augier.epub
getting epub: count-958916-Eve_au_corsage_fleuri_-_Jean_Sibil.epub
getting epub: count-994063-Le_Solliciteur_-_Eugene_Scribe.epub
getting epub: count-958911-Edgard_et_sa_bonne_-_Eugene_Labiche.epub
getting epub: count-959027-Ruy_Blas_-_Victor_Hugo.epub
getting epub: count-958988-Leonie_est_en_avance_-_Georges_Feydeau.epub
getting epub: count-994076-Cedrats_de_Sicile_-_Luigi_Pirandello.epub
getting epub: count-958974-Le_commissaire_est_bon_enfant_-_Georges_Courteline.e

KeyboardInterrupt: 